In [80]:
from gensim.models.keyedvectors import KeyedVectors
import numpy as np

import codecs
import argparse
import csv
import random

from collections import defaultdict

# very useful feature used to reload python modules
from importlib import reload

# import module that loads data, tokenises the tuples, initialises the embeddings matrix
import crim_data

import multiprojection_model
# contains code to evaluate according to semeval2018 metrics
import semeval_eval

from sklearn.model_selection import KFold

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [5]:
# initialise embeddings and normalise to unit-norm
model = KeyedVectors.load_word2vec_format('embeddings/GoogleNews-vectors-negative300.bin', binary=True)
#model = KeyedVectors.load_word2vec_format('embeddings/glove.42B.300d.txt', binary=False)
#model = KeyedVectors.load_word2vec_format('embeddings/wiki-news-300d-1M.vec', binary=False)

#model.save_word2vec_format('embeddings/GoogleNews-vectors-negative300.txt', binary=False)
model.init_sims(replace=True)

In [10]:
# import Ustalov et al. prepare module
import prepare

sub_train = prepare.read_subsumptions('subsumptions-train.txt')
sub_test = prepare.read_subsumptions('subsumptions-test.txt')
sub_validation = prepare.read_subsumptions('subsumptions-validation.txt')
synonyms = prepare.read_synonyms('synonyms.txt')  

# remove vocab term having no vector in embeddings
def get_terms_having_vectors(w2v, dataset):
    return [(q,h) for q, h in dataset if q in w2v and h in w2v]

sub_train = get_terms_having_vectors(model, sub_train)
sub_test = get_terms_having_vectors(model, sub_test)
sub_validation = get_terms_having_vectors(model, sub_validation)
synonyms = prepare.get_synonymys_having_vectors(synonyms, model)
synonyms.default_factory = None

# create hypernym dictionary
hyper_dict = defaultdict(list)
for x, y in sub_train + sub_test + sub_validation:        
    hyper_dict[x].append(y)
    
hyper_dict.default_factory = None

print ("Total number of tuples in entire set: %d" % (len([x for (x,y) in sub_train + sub_test + sub_validation])))
print ("Unique hyponyms in set: %d" % (len(set([x for (x,y) in sub_train + sub_test + sub_validation]))))

Total number of tuples in entire set: 7397
Unique hyponyms in set: 2259


In [ ]:
self.batch_size        = args['batch_size']
        self.phi_k             = args['phi_k']
        self.lambda_c          = args['lambda_c']
        self.epochs            = args['epochs']        
        self.negative_sample_n = args['negative_sample_n']
        self.synonym_sample_n  = args['synonym_sample_n']
        
        # this object generates the predictions from a model's learned parameters
        self.evaluator = args['evaluator']
        # this object scores the predictions according to MRR, MAP, P@k (k in {1,5,10,15})
        self.scorer = args['scorer']

In [82]:
args = {'w2v':model,
        'train':sub_train, 'test':sub_test, 'validation':sub_validation, 'synonyms':synonyms, 
        'limited_vocab_n': 250000,
        # model parameters        
        'batch_size': 32, 'phi_k': 1, 'lambda_c': 1., 'epochs': 10, 
        'negative_sample_n': 10, 'synonym_sample_n': 5
       }
#data = crim_data.CrimData(args)

In [89]:
args['data'] = data
args['evaluator'] = None
args['scorer'] = None

model = multiprojection_model.MultiProjModel(args)

{'epoch': [], 'loss': [], 'test_loss': []}

# Scratch Pad

In [88]:
#reload(crim_data)
reload(multiprojection_model)

<module 'multiprojection_model' from '/home/jovyan/work/hyperstar/multiprojection_model.py'>